In [ ]:
import numpy as np
from matplotlib import cm, pyplot as plt
import matplotlib.dates as dates
import pandas as pd
from datetime import timedelta,date

import sklearn

import warnings
warnings.filterwarnings("ignore")

import time

import pickle

from six import StringIO    #使用聚宽readfile函数


In [ ]:

para1_close_pd = pd.read_csv("para1_close.csv",index_col = 0)

para1_volume_pd = pd.read_csv("para1_volume.csv",index_col = 0)

para2_pd = pd.read_csv("para2.csv",index_col = 0)

In [ ]:


# 一、组成最基础的价格、成交额数据


################# 基础数据  ###################
# price_data_pd ： 从聚宽平台中提取的基础数据，表示沪深300指数股在时间段内的收盘价、成交额



# 沪深300指数股
# Secs = get_index_stocks('000300.XSHG')


# 存储最佳参数的变量赋初始值
Para_list = []
win_count = 0
total_count = 1
yield_rate = 0

# 二、搜集相应的数据进行数据的组装，生成收益率排名、成交额排名


#################  标准参数  #################
# 回滚观察过去的时间长度，表示以过去多长时间的长度作为观察长度
for n_period in range(2,20,4):

    
    # 每次获胜次数、总次数在每次循环时间归零
    win_count = 0
    total_count = 1
    yield_rate = 0

    print "check_point 0"

    #################### 生成收益率排名  ############
    # 收益率的pandas数组
    # data_yield_rank_pd ：表示以过去n_period为周期的时间段内的收益率排名，每天排名

    # 其中利用到全局参数 n_period
    data_yield_pd = pd.DataFrame()
    data_yield_pd = pd.rolling_sum(np.log(para1_close_pd/para1_close_pd.shift(1)),n_period)
    
    # 收益率的每天排序的数组
    # axis = 1：表示横向的数列进行排列
    # axis = 0：表示纵向的数列进行排列
    # 此处应该用1，采用横向的数组进行排列
    data_yield_rank_pd = data_yield_pd.rank(axis = 1)

    # 填充Nan数据为300
    data_yield_rank_pd.fillna(300,inplace = True)
    
    # 删除为Nan的行
    # data_yield_rank_pd = data_yield_rank_pd.dropna(axis=0,how='any')

    #################### 生成成交额排名  ############
    # data_money_rank_pd ：表示以过去n_period为周期的时间段内的成交额排名，每天排名
    data_money_pd = pd.DataFrame()
    data_money_pd = pd.rolling_sum(np.log(para1_volume_pd/para1_volume_pd.shift(1)),n_period)

    # 对数据做修补，删除TOP、BOTTOM数据
    # data_money_pd.drop(index = data_money_pd.nlargest(n = 20).index)
    
    # 收益率的每天排序的数组
    # axis = 1：表示横向的数列进行排列
    # axis = 0：表示纵向的数列进行排列
    # 此处应该用1，采用横向的数组进行排列
    data_money_rank_pd = data_money_pd.rank(axis = 1)

    # 填充Nan数据为300
    data_money_rank_pd.fillna(300,inplace = True)

    # 删除为Nan的行
    # data_money_rank_pd = data_money_rank_pd.dropna(axis=0,how='any')
    
    # print "check_point 1"
    
    
    
    #   三、按照co_XX的系数，结合两个排名pandas数组，最终生成打分的pandas数组
    
    # 收益率系数，用于加权
    for co_yield in range(-5,5):
        # 成交额系数，用于加权
        for co_money in range(-5,5):
            #################### 生成score_pd  ############
            # score_pd ： 按照成交额、收益率排名，乘以系数以后打分的得分值
            score_pd = pd.DataFrame()
            score_pd = data_yield_rank_pd*co_yield + data_money_rank_pd*co_money

            # 注意：因为是填充0，所以后面选择最佳组合数量时，推荐使用pd.nlargest(n=XXX)
            score_pd.fillna(0,inplace = True)
                        
        
            # 四、挑选股票，对比自选组合和市场组合的收益率
            #################### 关键参数 ##################
            # 延迟后n_lag天，延迟n_lag天作为最终的收益结算时间
            for n_lag in range(2,20,4):
                #################### 生成市场收益率（注意不是每日）  ############
                # market_yield_pd ：表示延迟n_lag天的收益率，用于与组合收益率进行比较
                market_data_pd = pd.DataFrame()
                
                market_data_pd = para2_pd
                
                # 注意：这里是滚动相加后再移位，移位根据测试需要加1
                market_yield_pd = pd.rolling_sum(np.log(para2_pd/para2_pd.shift(1)),n_lag).shift(-n_lag+1)

                # print "check_point 2"

                #################### 生成市场收益率（注意不是每日）  ############
                # portfolio_yield_pd ：表示延迟n_lag天的收益率，用于与组合收益率进行比较

                # 关键参数
                # 选择股票的数量
                n_length = 30

                # 每日的候选股票列表，结构为{日期：[股票1，股票2，股票3....股票n_length]}
                candidate_secs = {}

                
                
                for k,v in score_pd.iterrows():
                    # 注意：这里剔除了排名前20的最热股票，防止异常数据
                    candidate_secs[k] = list(v.nlargest(n=(n_length+20)).index[20:(n_length+20)])

                # print "check_point 3"

                # 合计所有数据的收益率
                # 相应组合的收益率从总的收益率中产生
                portfolio_yield_total_pd = pd.rolling_sum(np.log(para1_close_pd/para1_close_pd.shift(1)),n_lag).shift(-n_lag+1)


                # 用于暂存每个日期的收益率的字典
                portfolio_yield_dict = {}

                for i in range(0,portfolio_yield_total_pd.shape[0]):
                    portfolio_yield_dict[portfolio_yield_total_pd.index[i]] = sum(portfolio_yield_total_pd.loc[portfolio_yield_total_pd.index[i],candidate_secs[portfolio_yield_total_pd.index[i]]])/len(candidate_secs[portfolio_yield_total_pd.index[i]])
                # print "check_point 4"

                # 生成所需的组合收益率pandas数组
                portfolio_yield_pd = pd.DataFrame.from_dict(portfolio_yield_dict,orient='index')
                portfolio_yield_pd.columns = ['portfolio_yield']
                
                
                # 五、比较市场收益率与组合收益率的情况
                # 生成组合对比的pandas数组
                cmp_pd = pd.DataFrame()
                market_yield_pd.columns = ['market_yield']
                cmp_pd = pd.concat([market_yield_pd,portfolio_yield_pd],axis = 1)
                
                # 删除无效的Nan数据
                cmp_pd = cmp_pd.dropna(axis=0,how='any')

                # 计算portfolio大于market的数量
                win_count = cmp_pd[cmp_pd['portfolio_yield']>cmp_pd['market_yield']].shape[0]

                # 计算总的次数
                total_count = cmp_pd.shape[0]
                
                # 胜过市场的总的收益率
                yield_rate = sum(cmp_pd['portfolio_yield'] - cmp_pd['market_yield'])
                
                # print "找到参数，n_period为%d,co_yield为%d,co_money为%d,n_lag为%d,胜率为%f,收益率为%f"%(n_period,co_yield,co_money,n_lag,float(win_count)/float(total_count),yield_rate)
                
                # 判断是否达到标准条件
                if yield_rate> 0 :
                    if float(win_count)/float(total_count) > 0.6:
                        print "找到最佳参数，n_period为%d,co_yield为%d,co_money为%d,n_lag为%d,胜率为%f,收益率为%f"%(n_period,co_yield,co_money,n_lag,float(win_count)/float(total_count),yield_rate)
                        Para_list.append([n_period,co_yield,co_money,n_lag,win_count,total_count,yield_rate])


In [ ]:
para_best = pd.DataFrame(Para_list)
para_best.to_csv("best_para.csv")

从聚宽平台导出所需数据，并通过pickle模块另存为文件

In [14]:
# 模拟开始时间、结束时间
beginDate = '2014-1-1'
endDate = '2017-12-30'
Secs = get_index_stocks('000300.XSHG')

para1_pd = get_price(Secs, start_date = beginDate, end_date=endDate, frequency='daily', fields=['close','volume'],fq = "pre")


pickle.dump(para1_pd,open("para1.dat", 'wb'))


In [15]:
para2_pd = get_price('000300.XSHG', start_date = beginDate, end_date=endDate, frequency='daily', fields=['close'],fq = "pre")

pickle.dump(para2_pd,open("para2.dat", 'wb'))
